https://colab.research.google.com/drive/1pDYeniIB4wOrIsfHX_Kx_Z0FkUTbdrTq

In [0]:
!wget https://www.dropbox.com/s/ioqogaipt9i2hzo/Music_files.zip

--2019-10-02 19:28:30--  https://www.dropbox.com/s/ioqogaipt9i2hzo/Music_files.zip
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:6018:1::a27d:301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/ioqogaipt9i2hzo/Music_files.zip [following]
--2019-10-02 19:28:30--  https://www.dropbox.com/s/raw/ioqogaipt9i2hzo/Music_files.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1649d86a45c4c054824d1d0844.dl.dropboxusercontent.com/cd/0/inline/Apr0u9AWY8rAg2ns5hIjm-FGUCVqk-mlUw9dTvLItKsfftuOVxgGDhsKgU13Y9JFN-nt8UADnpA_Crfdl9ttKVj1E9qzhPGvddY1AcXx9FZOvqCr1vpxsRW7hxxzGoN_ddk/file# [following]
--2019-10-02 19:28:31--  https://uc1649d86a45c4c054824d1d0844.dl.dropboxusercontent.com/cd/0/inline/Apr0u9AWY8rAg2ns5hIjm-FGUCVqk-mlUw9dTvLItKsfftuOVxgGDhsKgU13Y9JFN-nt8UADnpA_Crfdl9ttKVj1E9qzhPGvddY1AcXx9

In [0]:
!unzip Music_files.zip

Archive:  Music_files.zip
  inflating: Music_files/Nintendo_-_Kirby_and_The_Amazing_Mirror_Area_1_Rainbow_Route_Piano_Cover_Play_2_Times.mid  
  inflating: Music_files/Nintendo_-_Kirby_and_The_Amazing_Mirror_Area_1_RainBow_Route_Piano_Cover_Play_50_Times.mid  
  inflating: Music_files/Nintendo_-_Kirby_and_The_Amazing_Mirror_Area_9_Candy_Constellation_Play_2_Times.mid  
  inflating: Music_files/Nintendo_-_Kirby_and_The_Amazing_Mirror_Area_9_Candy_Constellation_Play_50_Times.mid  
  inflating: Music_files/Nintendo_-_Kirby_Gourmet_Race_Round_1_and_3_(20Times).mid  
  inflating: Music_files/Nintendo_-_Kirby_Gourmet_Race_Round_1_and_3_(5_Times).mid  
  inflating: Music_files/Nintendo_-_Kirby_Gourmet_Race_Round_2_(20Times).mid  
  inflating: Music_files/Nintendo_-_Kirby_Nightmare_In_Dreamland_Kirby_Air_Grind_Play_3_Times.mid  
  inflating: Music_files/Nintendo_-_Kirby_Nightmare_Kirby_Air_Grind_Piano_Cover.mid  
  inflating: Music_files/Nintendo_-_Pokemon_Crystal_Gym_Play_3_Times.mid  
  infl

In [0]:
!ls

4570			     Music_files      sample_data
AABjLZ0vSQsBDRv8KZrKwb4Ia    Music_files.zip
AABjLZ0vSQsBDRv8KZrKwb4Ia.1  music.mid


In [0]:
!pip install mido
import mido
from mido import MidiFile, MidiTrack, Message
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import numpy as np

In [0]:
from music21 import converter, instrument, note, chord
#mid = MidiFile('/content/music.mid') 
notes = []
velocities = []

In [0]:
from glob import glob

In [0]:
for file in glob("/content/Music_files/*.mid"):
  print(file)
  midi = converter.parse(file)
  notes_to_parse = None
  parts = instrument.partitionByInstrument(midi)
  if parts: # file has instrument parts
      notes_to_parse = parts.parts[0].recurse()
  else: # file has notes in a flat structure
      notes_to_parse = midi.flat.notes
  for element in notes_to_parse:
      if isinstance(element, note.Note):
          notes.append(str(element.pitch))
      elif isinstance(element, chord.Chord):
          notes.append('.'.join(str(n) for n in element.normalOrder))

/content/Music_files/Nintendo_-_Kirby_and_The_Amazing_Mirror_Area_9_Candy_Constellation_Play_50_Times.mid
/content/Music_files/Nintendo_-_Pokemon_G_S_C_Goldenrod_City_Piano_Cover_Play_2_Times.mid
/content/Music_files/Nintendo_-_Pokemon_Fire_Red_Version_Gym_Theme_(Piano_Ver.).mid
/content/Music_files/Nintendo_-_Pokemon_FRLG_Battle_Gym_Leader_Play_2_Times.mid
/content/Music_files/Nintendo_-_Pokemon_Liquid_Crystal_Gym_Piano_Cover.mid
/content/Music_files/Nintendo_-_Kirby_Gourmet_Race_Round_1_and_3_(5_Times).mid
/content/Music_files/Nintendo_-_Kirby_and_The_Amazing_Mirror_Area_9_Candy_Constellation_Play_2_Times.mid
/content/Music_files/Nintendo_-_Pokemon_Fure_Red_Route_1_Play_3_Times.mid
/content/Music_files/Nintendo_-_Pokemon_Fire_Red_Pallet_Town_Theme_(2_Times).mid
/content/Music_files/Nintendo_-_Kirby_Gourmet_Race_Round_2_(20Times).mid
/content/Music_files/Nintendo_-_Pokemon_G_S_C_Bike_Ride_Theme_(3_Times).mid
/content/Music_files/Nintendo_-_Kirby_and_The_Amazing_Mirror_Area_1_Rainbow_R

In [0]:
from keras.utils import np_utils

In [0]:
len(notes)

218174

In [0]:
sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / np.max(network_input)
network_output = np_utils.to_categorical(network_output)

In [0]:
network_output.shape

(218074, 162)

In [0]:
np.max(network_input)

1.0

In [0]:
model = Sequential()
model.add(LSTM(
    256,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    return_sequences=True
))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dense(256))
model.add(Dropout(0.3))
model.add(Dense(network_output.shape[1]))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
model.fit(network_input, network_output, epochs=3, batch_size=256, verbose = 1)

Epoch 1/3
218074/218074 [==============================] - 728s 3ms/step - loss: 2.8474
Epoch 2/3
218074/218074 [==============================] - 726s 3ms/step - loss: 0.5272
Epoch 3/3
218074/218074 [==============================] - 724s 3ms/step - loss: 0.2177


In [0]:
start = np.random.randint(0, len(network_input)-1)
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
pattern = network_input[start].tolist()
prediction_output = []
# generate 500 notes
for note_index in range(500):
    prediction_input = np.reshape(pattern, (1, len(pattern), 1))
    prediction_input = prediction_input 
    prediction = model.predict(prediction_input, verbose=0)
    index = np.argmax(prediction)
    result = int_to_note[index]
    prediction_output.append(result)
    pattern.append([index/48])
    pattern = pattern[1:len(pattern)]

In [0]:
offset = 0
output_notes = []
# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [0]:
from music21 import stream

In [0]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='/content/test_output.mid')

'/content/test_output.mid'

In [0]:
from google.colab import files
files.download('/content/test_output.mid') 